# Standard Neural Networks Tutorial 1

This is a simple tutorial that will train the MNIST dataset using a standard neural network  
Let's first start by importing the librairies that we need and then print the TensorFlow and Keras versions.

In [0]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from PIL import Image, ImageOps

In [0]:
print("TensorFlow version: " + str(tf.__version__))
print("Keras version: " + str(tf.keras.__version__))

# Loading and Preprocessing

Let's start by loading the data from Keras dataset

The dataset is composed of:


*   60000 images for training.
*   10000 images for testing

Each image is a grayscale one of size 28x28.  
In order to use a standard NN to train this dataset, we will transform each image into a vector of dimension 28*28 = 784  
After this transformation the dimension will be:
* X: (?, 784)

The labels are numbers from 0 to 9 indicating which digit is in each image. The dimension is:
*   Y: (?, )

In [0]:
# Reading datasets
(x_train_orig, y_train_orig), (x_test_orig, y_test_orig) = load_data()


x_train = x_train_orig.reshape(x_train_orig.shape[0], x_train_orig.shape[1] * x_train_orig.shape[2]) / 255
x_test  = x_test_orig.reshape(x_test_orig.shape[0], x_test_orig.shape[1] * x_test_orig.shape[2]) / 255

print("Original X Train dimensions: " + str(x_train_orig.shape))
print("Transformed X Train dimensions: " + str(x_train.shape))
print("Y Train dimensions: " + str(y_train_orig.shape))

print("Original X Test dimensions: " + str(x_test_orig.shape))
print("Transformed X Test dimensions: " + str(x_test.shape))
print("Y Test dimensions: " + str(y_test_orig.shape))

In [0]:
# Printing a sample image
i=np.random.randint(0, x_train_orig.shape[0] - 1)
plt.title('Title: Plotting image:%d .... Value is %d' % (i,y_train_orig[i]))
#plt.imshow(np.squeeze(x_train_orig[i]), cmap='gray')
plt.imshow(x_train_orig[i], cmap='gray')
plt.show()

# Model

In this section we will use a model composed of only 2 layers:
- Layer 1 that has 100 units
- Layer 2 that has 10 units

Layer 1 takes a 784 dimension vector and produces a 100-dimension vector so its kernel has 78400 parameters and its bias 100 parameters => 78500

Layer 2 takes a 100-dimension vector and produces a 10-dimension vector so its kernel has 1000 parameters and its nias 10 => 1010

Overall, we have 79510 parameters  

__Note:__ We always need to specify the input tensor shape (without the batch size) in the first layer of our model. In this case the first Dense layer

In [0]:
model = Sequential()

model.add(Dense(100, activation='relu', input_shape=x_train.shape[1:]))
model.add(Dense(10, activation='softmax'))

# Compilation

Next we need to compile the model by calling __model.compile()__ and specifying the following:
* __Optimizer__ We can choose stochastic gradient descent __SGD__ or any other more powerful optimisation method like __Adam__
* __Loss function__ The sparse_categorical_crossentropy
* __Metrics__ to use while training. Here the accuracy between the real y and the predicted one  

__model.summary()__ summarises the model showing the layers along with their parameters

In [0]:
model.compile(optimizer="Adam", loss='sparse_categorical_crossentropy', metrics=["accuracy"])

model.summary()

# Model Training

We train the model by calling __model.fit()__ and giving it the training x_train and y_train_orig  
We can also validate the model by giving the x_test and y_test_orig

The model will be trained and the parameters will be updated based on x_train and y_train_orig **only**. x_test and y_test_orig will be used to evaluate how well the model is doing on the validation set

In [0]:
validation = True
if validation is True:
  eval_data = (x_test, y_test_orig)
else:
  eval_data = None

history = model.fit(x =x_train, y = y_train_orig, epochs=100, batch_size=1024, validation_data=eval_data)

# Training Results

In [0]:
print ("Train Accuracy = %.4f"  % (history.history['acc'][-1]))
print ("Train Loss = %.4f"  % (history.history['loss'][-1]))

if validation is True:
  print ("Test Accuracy = %.4f"  % (history.history['val_acc'][-1]))
  print ("Test Loss = %.4f"  % (history.history['val_loss'][-1]))

# Plotting Results

Here we plot the training and evaluation metrics' history.
2 Plots:
* One for loss
* One for accuracy

In [0]:
  plt.figure(figsize=(12,5))
  plt.subplot(1, 2, 1)
  plt.plot(history.history['acc'], 'b', label='Training acc')
  if validation is True:
    plt.plot(history.history['val_acc'], 'r', label='Validating acc')
  plt.title('Model Accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend()

  plt.subplot(1, 2, 2)
  plt.plot(history.history['loss'], 'b', label="Training loss")
  if validation is True:
    plt.plot(history.history['val_loss'], 'r', label="Validating loss")
  plt.title('Model Loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend()

  plt.show()

In [0]:
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=-1)

np.where(y_pred != y_test_orig)

In [0]:
i=9944
print("Original:%d\tPredicted:%d" % (y_test_orig[i], y_pred[i]))
plt.title('Title: Plotting image '+str(i))
plt.imshow(x_test_orig[i], cmap='gray')
plt.show()

# Predict 

Prepare an image of a digit between 0 and 9. Let the digit be in black on a white background.
Change the name of the file in __imgname__ variable to be the full path to the image that you have prepared.  

If you are running this notebook in Google Colab and the image does not exist, it will prompt you to upload it.  
If you are running it locally, it will just prompt an error message telling you that the file is not found.

In [0]:
import os

try:
  from google.colab import files
except ImportError:
  pass

imgname = 'digit-7.png'

if os.path.exists(imgname) == False:
  try:
    files.upload()
  except:
    pass

try:
  # Read Image, convert it to grayscale, resize it to 28x28, invert to in order to be white on black background
  img = Image.open(imgname).convert('L')
  img = img.resize((28,28))
  img = ImageOps.invert(img)

  # Transform it to an array, normalise it and then convert it to a vector
  myimg = np.array(img)
  myimg = myimg.reshape((1, myimg.shape[0] * myimg.shape[1])) / 255

  # Predict
  pred = model.predict(myimg)
  pred = np.argmax(pred, axis=-1)

  print("Predicted:%d" % (pred))
  plt.title('Title: Plotting image ')
  plt.imshow(img, cmap='gray')
  plt.show()
except:
  print("File not found!!!")

# Summary
In this tutorial, we learnt how to:
* Load and Preprocess data
* Create a Sequential Model
* Compile and view the model
* Train
* Visualise training results
* Predict an uploaded image